<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_holds_info_for_patron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="250"/>

# REST API: Lookup Holds Info Given Patron Record Number

---


## 1. Enter REST-API Credentials

In order to be authorized to use the REST API, you must enter valid credientials.

When prompted, enter in the **`auth_string`** as your password:

In [1]:
# <--- Click the play button to the left to get started

# this is the helper script that contains some needed functions
!wget --quiet https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/master/misc/chpl_helper.py --output-document=chpl_helper.py

# import the helper functions we just downloaded ...
import chpl_helper
import getpass
from google.colab import files
import pandas as pd
import requests
import json
import re

# this is to grab the record number from the end of a response
re_record_num = re.compile('.*?([0-9]+)$')

# point this to the base URL for the Sierra REST API
base_url = 'https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/'

# client_key = getpass('Enter the client KEY value: ')
# client_secret = getpass('Enter the client SECRET value: ')
auth_string = getpass.getpass('Enter the auth_string value: ')

# test our credentials

headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)
r = requests.get(base_url + 'info/token', headers=headers, verify=True)
print('access token expires in: {} seconds'.format(r.json()['expiresIn']))

Enter the auth_string value: ··········
access token expires in: 3599 seconds


## 2. Input patron record number and get information on holds

Note, There is no `p` or checkdigit on this record number--*just the number portion*

In [2]:
patron_record_num = input('patron record num: ')

try:
  r = requests.get(base_url + f'patrons/{patron_record_num}/holds' , headers=headers, params={'fields': 'pickupByDate'})

  print('total number of holds: ', r.json()['total'], '\n---\n', sep='', end='')

  if r.json()['total'] > 0:
    for i, entry in enumerate(r.json()['entries'], start=1):
      # if 'id' in entry:
      #   print(entry['id'], end=' ')
      r_hold = requests.get(entry['id'], headers=headers, params={'fields': 'placed,pickupLocation,pickupLocation,note,record,status'})
      # print(i, r_hold.json(), end='')
      m = re_record_num.match(r_hold.json()['id'])
      hold_num = m[1] if m else ''
      m = re_record_num.match(r_hold.json()['record'])
      hold_record_num = m[1] if m else ''
      print(
          i, ' ',
          'hold id: ', hold_num, 
          'for record: ', r_hold.json()['record'],
          'status: ', r_hold.json()['status'], 
          'placed: ', r_hold.json()['placed'],
          'pickupLocation: ', r_hold.json()['pickupLocation'],
          'note: ', r_hold.json()['note'],
          end=''
      )

      if 'pickupByDate' in entry:
        print('pickupByDate: ', entry['pickupByDate'])
      else:
        print('')

except:
  pass



patron record num: 2509457
total number of holds: 30
---
1   hold id:  50089033 for record:  https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1794877 status:  {'code': '0', 'name': 'on hold.'} placed:  2022-01-31 pickupLocation:  {'code': 'dt', 'name': 'Delhi Township'} note:  chpl-bulk-hold-script
2   hold id:  50089044 for record:  https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/10169651 status:  {'code': '0', 'name': 'on hold.'} placed:  2022-01-31 pickupLocation:  {'code': 'dt', 'name': 'Delhi Township'} note:  chpl-bulk-hold-script
3   hold id:  50089045 for record:  https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/3745043 status:  {'code': '0', 'name': 'on hold.'} placed:  2022-01-31 pickupLocation:  {'code': 'dt', 'name': 'Delhi Township'} note:  chpl-bulk-hold-script
4   hold id:  50089046 for record:  https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/4075716 status:  {'code': '0', 'name': 'on hold.'} placed:  2022-01-31 picku